In [1]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from stacked_generalizer import StackedGeneralizer
import numpy as np

In [2]:
import numpy as np
import pandas as pd
import os
os.chdir("/Users/shuozhang/Desktop/data")
df=pd.read_csv('lmmodel.csv', sep='\t', encoding='utf-8')
df.drop(['Unnamed: 0'], inplace=True, axis=1)

In [3]:
df.head()

,Month,Weekday,Hour,Temp,Rain,count,10001,10002,10003,10004,...,11428,11429,11430,11432,11433,11434,11435,11436,11439,11451
0,4,1,0,49.0,0.0,375,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,1,0,49.0,0.0,332,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,1,0,49.0,0.0,521,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,0,49.0,0.0,49,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,4,1,0,49.0,0.0,55,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
target=df[['count']]

In [5]:
data=df[[col for col in df.columns if col not in ['count']]]

In [6]:
import sklearn.cross_validation as cv
x_train, x_test, y_train, y_test = cv.train_test_split(data, target, 
  test_size=2.0/10, random_state=0)

In [7]:
print x_train.shape
print x_test.shape

(490838, 203)
(122710, 203)


In [86]:
XGB=xgb.XGBRegressor(max_depth=14,learning_rate=0.1186,n_estimators=463,silent=True,
            nthread=-1,gamma=1.0,min_child_weight=6.1929,subsample=0.9675,colsample_bytree=0.8544)

In [87]:
XGB.fit(x_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8544,
       gamma=1.0, learning_rate=0.1186, max_delta_step=0, max_depth=14,
       min_child_weight=6.1929, missing=None, n_estimators=463, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.9675)

In [8]:
RFR=RandomForestRegressor(max_features=14,n_estimators=300)

In [9]:
RFR.fit(x_train, y_train)

/Users/shuozhang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=14, max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=300, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [39]:
pred_y_train=RFR.predict(x_train)

In [43]:
pred_y_train=pd.DataFrame(pred_y_train, columns=['pred_y_trainrf'])

In [44]:
pred_y_test=RFR.predict(x_test)

In [47]:
pred_y_test=pd.DataFrame(pred_y_test, columns=['pred_y_testrf'])

In [45]:
pred_y_train.to_csv('pred_y_trainrf.csv', sep='\t')

In [48]:
pred_y_test.to_csv('pred_y_testrf.csv', sep='\t')

In [49]:
x_train.to_csv('x_trainrf.csv', sep='\t')
x_test.to_csv('x_testrf.csv', sep='\t')

In [50]:
y_train.to_csv('y_trainrf.csv', sep='\t')

In [51]:
y_test.to_csv('y_testrf.csv', sep='\t')

In [53]:
x_train.shape

(490838, 203)

In [10]:
pred_y_trainxgb=pd.read_csv('pred_y_trainxgb.csv', sep='\t', index_col=0)

In [11]:
pred_y_train=pd.read_csv('pred_y_trainrf.csv', sep='\t', index_col=0)

In [12]:
pred_y_test=pd.read_csv('pred_y_testrf.csv', sep='\t', index_col=0)

In [13]:
pred_y_train.head()

,pred_y_trainrf
0,12.943333
1,523.213333
2,3.213889
3,60.580833
4,1.320000


In [14]:
pred_y_trainxgb.head()

,pred_y_train
0,10.542413
1,520.186279
2,2.436053
3,62.625908
4,0.887218


In [15]:
pred_y_com=pd.concat([pred_y_train,pred_y_trainxgb], axis=1)

In [16]:
pred_y_com.head()

,pred_y_trainrf,pred_y_train
0,12.943333,10.542413
1,523.213333,520.186279
2,3.213889,2.436053
3,60.580833,62.625908
4,1.320000,0.887218


In [17]:
y_train.index=np.arange(y_train.shape[0])

In [18]:
pred_y_com1=pd.concat([y_train,pred_y_com], axis=1)

In [19]:
pred_y_com1.head()

,count,pred_y_trainrf,pred_y_train
0,13,12.943333,10.542413
1,544,523.213333,520.186279
2,4,3.213889,2.436053
3,58,60.580833,62.625908
4,1,1.320000,0.887218


In [20]:
from sklearn import linear_model
ols = linear_model.LinearRegression(fit_intercept=False)

In [21]:
ols.fit(pred_y_com, y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=False)

In [22]:
ols.score(pred_y_com, y_train)

0.99616130184638141

In [23]:
pred_ytrain_com=ols.predict(pred_y_com)

In [26]:
from sklearn.metrics import mean_squared_error
import math
math.sqrt(mean_squared_error(y_train, pred_ytrain_com))

15.738548299150535

In [27]:
ols.intercept_

0.0

In [28]:
ols.coef_

array([[ 1.22234424, -0.21395961]])

In [29]:
pred_y_testxgb=pd.read_csv('pred_y_testxgb.csv', sep='\t', index_col=0)

In [30]:
pred_y_test.head()

,pred_y_testrf
0,13.402667
1,11.606667
2,971.086667
3,2.387500
4,1.254444


In [31]:
pred_y_testcom=pd.concat([pred_y_test,pred_y_testxgb], axis=1)

In [32]:
pred_y_testcom.head()

,pred_y_testrf,pred_y_test
0,13.402667,14.319020
1,11.606667,12.530326
2,971.086667,973.600525
3,2.387500,3.174253
4,1.254444,1.378159


In [33]:
pred_y_test.columns=['pred_y_RFR']

In [34]:
pred_y_testxgb.columns=['pred_y_XGB']

In [35]:
pred_y_ensemble=ols.predict(pred_y_testcom)

In [36]:
pred_y_ensemble=pd.DataFrame(pred_y_ensemble, columns=['pred_y_ensemble'])

In [37]:
y_test.columns=['true value']

In [38]:
y_test.index=np.arange(y_test.shape[0])

In [39]:
pred_y_final=pd.concat([y_test, pred_y_test,pred_y_testxgb, pred_y_ensemble], axis=1)

In [74]:
pred_y_final.head()

,true value,pred_y_RFR,pred_y_XGB,pred_y_ensemble
0,17,13.402667,14.319020,13.318980
1,13,11.606667,12.530326,11.506359
2,961,971.086667,973.600525,978.691008
3,3,2.387500,3.174253,2.239185
4,3,1.254444,1.378159,1.238493


In [75]:
pred_y_final1=pred_y_final.round(0)

In [76]:
pred_y_final1=pred_y_final1.applymap(np.int)

In [77]:
pred_y_final1[pred_y_final1<0]=0

In [78]:
pred_y_final1.head()

,true value,pred_y_RFR,pred_y_XGB,pred_y_ensemble
0,17,13,14,13
1,13,12,13,12
2,961,971,974,979
3,3,2,3,2
4,3,1,1,1


In [79]:
pred_y_final1.to_csv('pred_y_final1.csv', sep='\t')

In [45]:
from sklearn.metrics import mean_squared_error
import math
math.sqrt(mean_squared_error(y_test, pred_y_ensemble))

42.94675196878784

In [46]:
pred_y_final1 = pred_y_final1.sort(['true value'], ascending=0)

/Users/shuozhang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [47]:
pred_y_final1.head()

,true value,pred_y_RFR,pred_y_XGB,pred_y_ensemble
83148,2328,2107,2130,2120
112966,2305,1730,1979,1691
100549,2264,1522,1934,1447
74122,2263,1928,2133,1900
5573,2218,1881,2046,1861


In [48]:
math.sqrt(mean_squared_error(pred_y_final1['pred_y_RFR'],pred_y_final1['pred_y_XGB'] ))

20.331401502374845

In [331]:
pred_y_final1.index=np.arange(pred_y_final1.shape[0])

In [338]:
rows = np.random.choice(cut1.index.values, 10)
sample = cut1.ix[rows]

In [339]:
sample1=sample.sort(['true value'], ascending=0)

/Users/shuozhang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [340]:
sample1

,true value,pred_y_RFR,pred_y_XGB,pred_y_ensemble
79,1885,1909,1873,1933
139,1774,1596,1608,1607
408,1529,1459,1524,1457
983,1261,1061,952,1093
990,1260,1289,1283,1302
1029,1247,1038,1217,1008
1196,1205,979,1007,982
1694,1110,1022,1079,1018
2008,1063,924,999,915
2155,1044,1063,1046,1075


In [ ]:
import pylab as plt
import numpy as np
x = np.arange(1, 11)
plt.plot(x, sample1['true value'], label='true value')
plt.plot(x, sample1['pred_y_RFR'], label='RFR')
plt.plot(x, sample1['pred_y_XGB'], label='XGB')
plt.plot(x, sample1['pred_y_ensemble'], label='Ensemble')
plt.legend()
plt.xlabel('Sample')
plt.ylabel('Taxi Demand')
plt.title('Comparison of different models')
plt.show()

In [148]:
sample1.to_csv('sample1.csv', sep='\t', index_col=0)
sample2.to_csv('sample2.csv', sep='\t', index_col=0)
sample3.to_csv('sample3.csv', sep='\t', index_col=0)

In [143]:
rows = np.random.choice(cut2.index.values, 10)
sample2 = cut2.ix[rows]
sample2=sample2.sort(['true value'], ascending=0)

/Users/shuozhang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [145]:
rows = np.random.choice(cut3.index.values, 10)
sample3 = cut3.ix[rows]
sample3=sample3.sort(['true value'], ascending=0)

/Users/shuozhang/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


In [335]:
cut1=pred_y_final1[pred_y_final1['true value']>=1000]

In [124]:
print math.sqrt(mean_squared_error(cut1['true value'], cut1['pred_y_RFR']))
print math.sqrt(mean_squared_error(cut1['true value'], cut1['pred_y_XGB']))
print math.sqrt(mean_squared_error(cut1['true value'], cut1['pred_y_ensemble']))

150.122432218
122.937947793
156.256575497


In [130]:
cut1.shape

(2479, 4)

In [336]:
cut2=pred_y_final1[(pred_y_final1['true value']<1000) & (pred_y_final1['true value']>=100)]

In [129]:
print math.sqrt(mean_squared_error(cut2['true value'], cut2['pred_y_RFR']))
print math.sqrt(mean_squared_error(cut2['true value'], cut2['pred_y_XGB']))
print math.sqrt(mean_squared_error(cut2['true value'], cut2['pred_y_ensemble']))

75.1058926373
66.0370688524
79.6853986182


In [131]:
cut2.shape

(24759, 4)

In [337]:
cut3=pred_y_final1[pred_y_final1['true value']<100]

In [133]:
print math.sqrt(mean_squared_error(cut3['true value'], cut3['pred_y_RFR']))
print math.sqrt(mean_squared_error(cut3['true value'], cut3['pred_y_XGB']))
print math.sqrt(mean_squared_error(cut3['true value'], cut3['pred_y_ensemble']))

8.67826721975
7.15509738243
9.48695663728


In [134]:
cut3.shape

(95472, 4)

In [113]:
pred_y_final1.to_csv('pred_y_final1.csv', sep='\t')

In [159]:
import pylab as plt
import numpy as np
x = np.arange(1, 11)
plt.plot(x, sample2['true value'], label='true value')
plt.plot(x, sample2['pred_y_RFR'], label='RFR')
plt.plot(x, sample2['pred_y_XGB'], label='XGB')
plt.plot(x, sample2['pred_y_ensemble'], label='Ensemble')
plt.legend()
plt.xlabel('Sample')
plt.ylabel('Taxi Demand')
plt.title('Comparison of different models')
plt.show()

In [160]:
import pylab as plt
import numpy as np
x = np.arange(1, 11)
plt.plot(x, sample3['true value'], label='true value')
plt.plot(x, sample3['pred_y_RFR'], label='RFR')
plt.plot(x, sample3['pred_y_XGB'], label='XGB')
plt.plot(x, sample3['pred_y_ensemble'], label='Ensemble')
plt.legend()
plt.xlabel('Sample')
plt.ylabel('Taxi Demand')
plt.title('Comparison of different models')
plt.show()

In [49]:
seppred=pd.read_csv('predicitondatafinal.csv', sep='\t', index_col=0)

In [50]:
seppred.shape

(33264, 203)

In [51]:
pred_sep=RFR.predict(seppred)

In [55]:
pred_sep_RFR=pd.DataFrame(pred_sep, columns=['pred_RFR'])

TypeError: __init__() got an unexpected keyword argument 'index_col'

In [58]:
pred_sep_XGB=pd.read_csv('xgb_predict.csv',sep='\t', index_col=0 )

In [59]:
pred_sep_com=pd.concat([pred_sep_RFR,pred_sep_XGB], axis=1)

In [60]:
pred_sep_com.head()

,pred_RFR,xbg_predict
0,2.433333,-2.358335
1,4.685000,-7.960156
2,5.331667,-4.649641
3,8.323333,2.053402
4,6.106667,0.944294


In [61]:
pred_sep_ensemble=ols.predict(pred_sep_com)

In [62]:
pred_sep_com['pred_ensemble']=pred_sep_ensemble

In [63]:
pred_sep_com.head()

,pred_RFR,xbg_predict,pred_ensemble
0,2.433333,-2.358335,3.478959
1,4.685000,-7.960156,7.429835
2,5.331667,-4.649641,7.511967
3,8.323333,2.053402,9.734633
4,6.106667,0.944294,7.262408


In [64]:
pred_sep_com[pred_sep_com < 0] = 0

In [65]:
pred_sep_com=pred_sep_com.round(0)

In [66]:
pred_sep_com.shape

(33264, 3)

In [67]:
preddata=pd.read_csv('predicitondata.csv', sep='\t', index_col=0)

In [68]:
preddata.shape

(33264, 6)

In [69]:
shinydata=pd.concat([preddata,pred_sep_com], axis=1)

In [73]:
shinydata

,Month,Weekday,Hour,zipcode,Temp,Rain,pred_RFR,xbg_predict,pred_ensemble
0,9,0,0,10270,19,0,2.0,0.0,3.0
1,9,0,0,10275,19,0,5.0,0.0,7.0
2,9,0,0,10278,19,0,5.0,0.0,8.0
3,9,0,0,10280,19,0,8.0,2.0,10.0
4,9,0,0,10281,19,0,6.0,1.0,7.0
5,9,0,0,10282,19,0,11.0,1.0,14.0
6,9,0,0,10301,19,0,5.0,0.0,7.0
7,9,0,0,10302,19,0,5.0,0.0,8.0
8,9,0,0,10303,19,0,5.0,0.0,7.0
9,9,0,0,10304,19,0,5.0,0.0,7.0


In [72]:
shinydata.to_csv('shinydata.csv', sep='\t')